In [20]:
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import datetime
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data
import numpy as np
import quandl
import math  

In [21]:
currentDate = datetime.datetime.now()
startDate = datetime.datetime(2013, 5, 28)
endDate = datetime.datetime(currentDate.year, currentDate.month, currentDate.day)
dateRange = pd.date_range(startDate, endDate)

In [22]:
# Define variables
initial_investment = 25000
btc = "BTC-USD"
eth = "ETH-USD"
xrp = "XRP-USD"
ltc = "LTC-USD"
sym = [btc, xrp, ltc, eth]

In [23]:
df_btc = data.DataReader(btc, 'yahoo', startDate);
df_btc.drop(columns=["High", "Low", "Open", "Close", "Volume"], inplace=True)

df_ltc = data.DataReader(ltc, 'yahoo', startDate);
df_ltc.drop(columns=["High", "Low", "Open", "Close", "Volume"], inplace=True)

df_xrp = data.DataReader(xrp, 'yahoo', startDate);
df_xrp.drop(columns=["High", "Low", "Open", "Close", "Volume"], inplace=True)

df_eth = data.DataReader(eth, 'yahoo', startDate);
df_eth.drop(columns=["High", "Low", "Open", "Close", "Volume"], inplace=True)

ConnectionError: HTTPConnectionPool(host='ichart.finance.yahoo.com', port=80): Max retries exceeded with url: /table.csv?s=BTC-USD&a=4&b=28&c=2013&d=5&e=15&f=2019&g=d&ignore=.csv (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000188BB40FE48>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [ ]:
#Save df with only Adj Close value
df = df_btc.join([df_eth, df_ltc,df_xrp], how="inner")
df.columns=[f"{btc}_Adj Close", f"{eth}_Adj Close", f"{ltc}_Adj Close", f"{xrp}_Adj Close"]

In [ ]:
symbols = [df_btc, df_xrp, df_ltc, df_eth]
for crypto_df in (symbols):
    crypto_df['norm_return'] = crypto_df['Adj Close'] / crypto_df.iloc[0]['Adj Close']

In [ ]:
#Choosing portfolio allocation. 
# 10% BTC
# 20% xrp
# 30% LTC
# 40% ETH

In [ ]:
# allocation the percentage with the symbols
allo = [.1, .2, .3, .4]
for crypto_df , crypto in zip(symbols, allo):
    crypto_df["Portfolio_alloc"] = crypto_df["norm_return"] * crypto

In [ ]:
# Compute prosition values based on the initialinvestement
for crypto_df in symbols:
    crypto_df["Pos_value"] = crypto_df["Portfolio_alloc"] * initial_investment


In [ ]:
df_btc.head()

In [ ]:
# Rename each df position value name
df_btc.rename(columns={"Pos_value": f"{btc}_Pos_value"}, inplace=True)
df_ltc.rename(columns={"Pos_value": f"{ltc}_Pos_value"}, inplace=True)
df_xrp.rename(columns={"Pos_value": f"{xrp}_Pos_value"}, inplace=True)
df_eth.rename(columns={"Pos_value": f"{eth}_Pos_value"}, inplace=True);

In [ ]:
# Create position value list and join to the porfolio
df_btc.drop(["Adj Close","norm_return","Portfolio_alloc"], axis=1, inplace=True)
df_ltc.drop(["Adj Close","norm_return","Portfolio_alloc"], axis=1, inplace=True)
df_eth.drop(["Adj Close","norm_return","Portfolio_alloc"], axis=1, inplace=True)
df_xrp.drop(["Adj Close","norm_return","Portfolio_alloc"], axis=1, inplace=True)

crypto_portfolio = df_btc.join([df_eth, df_ltc,df_xrp], how="inner")

In [ ]:
#Compute total position 
crypto_portfolio["Total Port Pos"] = crypto_portfolio.sum(axis=1)

In [ ]:
crypto_portfolio.head()

In [ ]:
#Plot total portfolio
crypto_portfolio["Total Port Pos"].plot(figsize=(10,8));

In [ ]:
print("If I invested", initial_investment, "with the following portfolioof ", "10% BTC, 20% xrp, 30% LTC, 40% ETH", '\n')
print("I would have earned", crypto_portfolio["Total Port Pos"].max(), "in", crypto_portfolio["Total Port Pos"].idxmax());


In [ ]:
#Plot indvidual position changes over time
crypto_portfolio.drop("Total Port Pos", axis=1).plot(figsize=(10,8));

In [ ]:
# Notice that ETH grew the most as expected becuase 40% was invested in ETH

In [ ]:
# Check how LTC and ETH perform with respect to BTC
# Alpha value tells us how well one preform over the other bigger alpha
# beta tells us how the other crypto moves with respect to BTC
# greater beta means it is more reactive
beta_ltc, alpha_ltc = np.polyfit(df[f"{btc}_Adj Close"], df[f"{ltc}_Adj Close"], 1)
print("Beta LTC",beta_ltc)
print("Alpha LTC",alpha_ltc, '\n')

beta_eth, alpha_eth = np.polyfit(df[f"{btc}_Adj Close"], df[f"{eth}_Adj Close"], 1)
print("Beta ETH",beta_eth)
print("Alpha ETH",alpha_eth, '\n')

beta_xrp, alpha_xrp = np.polyfit(df[f"{btc}_Adj Close"], df[f"{xrp}_Adj Close"], 1)
print("Beta XRP",beta_xrp)
print("Alpha XRP",alpha_xrp, '\n')

In [24]:
# Now we perform more statiscal computation to optimze our portfolio
#Compute Daily Return of the total portfolio position. Notice the first Daily return is nan since we don't data previous data
crypto_portfolio["Daily Return"] = crypto_portfolio["Total Port Pos"].pct_change(1)

NameError: name 'crypto_portfolio' is not defined

In [ ]:
# Compute mean or average daily return 
crypto_portfolio["Daily Return"].mean()

In [ ]:
# Compute standard deviation of the daily return

crypto_portfolio["Daily Return"].std()

In [ ]:
# Compute the histrogram for the daily return
crypto_portfolio["Daily Return"].plot(kind='hist', figsize=(9,5), bins=100);

In [ ]:
# Compute the overal comulatuve daily return
cum_return = (1+ crypto_portfolio["Daily Return"]).cumprod()
cum_return.plot(figsize=(9,5));

In [ ]:
# Compute total comulative return
cum_return_total = 100 * (crypto_portfolio["Total Port Pos"][-1]/ crypto_portfolio["Total Port Pos"][0]-1)
cum_return_total = str(round(cum_return_total, 2))
print("The total cumalative return for the ${} is: {}%".format(initial_investment,cum_return_total))

In [ ]:
crypto_portfolio["Total Port Pos"][-1] = str(round(crypto_portfolio["Total Port Pos"][-1], 2))
print("With a {}% return of ${}, our total return is ${}".format(cum_return_total, initial_investment, crypto_portfolio["Total Port Pos"][-1]))

In [ ]:
# To help find out how to measure the risk adjusted retrun, we compute the sharp ratio
# Which is the mean minus the risk free rate divided by the sdt
# We will assume a Risk free rate of zero for crypto.
SR = crypto_portfolio["Daily Return"].mean() / crypto_portfolio["Daily Return"].std()
# With a daily Sample rate, we can compute the final value of Annu_SR = K * SR.
Annu_SR = math.sqrt(252) * SR

In [ ]:
print("The shrap ratio of our portfolio is {} which is greater than 1".format(str(round(Annu_SR,2))))
print("In general many investers consider a SR greater than 1 to be a good portfolio")

# Optimizing our Portfolio
### Instead of using Monte Carlos Simulation method to do a try and guess, we will be using Scipy build in optimzation using the Sharp Ratio
### We will also use the Adj Close values computed previously

In [ ]:
# Computer the log return instead of the pct_change
log_return = np.log(df/df.shift(1))

In [ ]:
# Compute a function that will return the expected return, the expected volatility nd the SR
def Return_SR_Vol_Retrun(weight):
    # Compute random weight that will be associate with the crypto symbols
    weight = np.array(np.random.random(4))
    # To normalize the weight we compute the folowing
    weight = weights/np.sum(weight)
    # Compute the expected return over 252 days
    expect_return = np.sum((log_return.mean()* weights) * 252)
    # Compute the expected variance or volatility over 252 days
    expect_volality = np.sqrt(np.dot(weights.T, np.dot(log_return.cov() * 252, weights)))
    # Compute the SR or Sharp Ration
    SR_daily = expect_return/expect_volality
    return np.array([expect_return, expect_volality, SR_daily])
    

In [ ]:
# Compute helper function to use the scipy minimizer and a function to return the sum
def Get_neg_SR(weights):
    return Return_SR_Vol_Retrun(weights)[3] * -1

def Get_Sum(weights):
    if(np.sum(weights) == 1):
        return 0
    else:
        return np.sum(weights)

In [ ]:
# Initialize the minimizer parameters
bound = ((0.1), (0.1), (0.1), (0.1))
constraint = {'type':'eq', 'fun': Get_Sum}
guesses = [0.15, 0.15, 0.15, 0.15]

minimize_result = minimize(Get_neg_SR, guesses, method='SLSQP', bounds= bound, constraints=constraint)

In [ ]:
Get_neg_SR(minimize_result.x) # Return expect_return, expect_volality, SR_daily respectivelu 

In [ ]:
## Using efficient frontier which help us find the lowesr rsk possible for a certain expected return given
efficient_front = np.linspace(0, 0.3, 100)
def Mini_Voll(weights):
    return Get_neg_SR(weights)[1]

efficient_front_Vol = []
for item in efficient_front:
    constraint = {'type':'eq', 'fun': Get_Sum}, {'type':'eq', 'fun': lambda w: Get_neg_SR(w)[0]-item}
    minimize_value = minimize(Mini_Voll, guesses, method='SLSQP', bounds= bound, constraints=constraint)
    efficient_front_Vol.append(minimize_value["fun"])

In [ ]:
#Plotting the SR optimization
ex_ret = np.sum((log_return.mean()* weights) * 252)
exp_vol = np.sqrt(np.dot(weights.T, np.dot(log_return.cov() * 252, weights)))
sr_da = expect_return/expect_volality
plt.figure(figsize=(12,8))
plt.scatter(exp_vol, ex_ret, c= sr_da, cmap='plasma')
plt.plot(efficient_front_Vol, 'g--', linewidth=4)

In [26]:
#if a customer say he or she wants a max volatilty of 0.25 then you can look at the plot and provide the best return percentage based on the volality.
#Basically the plot lets you pick the best possible return rate based on the rist of volatily choosen